In [1]:
import os
import modules.util as util
import math
import numpy as np
import modules.datagenerator as dtgen
import modules.network_PoolerCLS as network
import modules.loss as loss
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow import keras
import random

2023-03-31 15:21:47.177696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
random.seed(15)#fixed random state 
np.random.seed(15)
tf.random.set_seed(15)

home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_testset')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [3]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'test': np.arange(math.floor(len(pairs)))}

# Generators
# test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=16)
test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=1)

In [4]:
print(f"Number of Test Pairs: {len(partition['test'])}")

Number of Test Pairs: 1262


In [5]:
# !pip list Total 8260

In [6]:
# from transformers import TFViTModel
# TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [7]:
siamese_obj = network.SiameseNetwork()
siamese_network = siamese_obj.make_siamese_net()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [8]:
siamese_network.compile(loss= loss.loss(1), optimizer=tf.keras.optimizers.Adam(learning_rate=0.5e-3), # 0.5e-6
                    metrics=["accuracy"])

In [11]:
# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Unrestored model, accuracy: {:5.2f}%".format(100 * acc))


1320/1320 - 76s - loss: 0.2572 - accuracy: 0.5174 - 76s/epoch - 58ms/step
Unrestored model, accuracy: 51.74%


In [9]:
checkpoint_path = os.path.join(home, 'checkpoint')

# Loads the weights
siamese_network.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

2023-03-31 15:22:10.306849: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /home/halim/Documents/ECE613/GitHub/CC_results_samplew/18hr_SSIM_SSIMMOD/checkpoint: FAILED_PRECONDITION: /home/halim/Documents/ECE613/GitHub/CC_results_samplew/18hr_SSIM_SSIMMOD/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


1262/1262 [==============================] - 68s 48ms/step - loss: 0.2621 - accuracy: 0.6569
Restored model, accuracy: 65.69%


In [9]:
predictions = siamese_network.predict(test_generator)

1320/1320 [==============================] - 78s 55ms/step


In [10]:
predictions.shape

(1320, 1)

In [11]:
test_generator.labels

[['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp',
  0,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-042/W-042-24.bmp',
  1,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  0,
  70,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2mar

In [12]:
labels = []
for i in test_generator.labels:
    # print(i[2])
    labels.append(i[2])

In [13]:
len(labels)

1320

In [14]:
predictions.shape

(1320, 1)

In [15]:
predictions[3][0]

1.0

In [16]:
pred_lbl=[]
for item in predictions:
    print(item[0])
    pred_lbl.append(1 if item[0]>=0.5 else 0)

0.63557726
0.99397546
1.0
1.0
0.4335605
0.018802434
0.686775
0.9803469
0.9653237
0.99902284
0.9841287
0.9999957
0.8310612
0.9999858
1.0
1.0
0.980706
0.9215514
0.4335605
0.9999989
0.62341243
0.92893314
0.9986173
0.99999285
0.055357523
0.22974396
1.0
1.0
0.12386103
0.99989426
1.0
0.99999714
0.12307798
0.9999999
0.9900671
0.89739114
0.99937326
0.99991524
0.28353179
0.99999976
0.7060375
0.9928348
0.7060375
0.9999982
1.0
1.0
0.3610052
0.8903099
0.78107965
0.99808633
0.018270375
0.9995284
0.034208216
0.99998915
0.9983999
0.310277
0.706442
0.9999912
0.596901
0.99997056
0.71928275
0.99999654
0.37841743
0.8090383
0.99965394
0.99968266
0.8228218
0.99091786
0.68883103
0.87197983
1.0
1.0
0.8541665
0.98882955
0.9775454
0.99745613
1.0
1.0
0.06469358
0.7972277
1.0
1.0
0.2879716
0.9851568
0.8714258
1.0
0.3526997
0.99843794
1.0
1.0
0.8760134
1.0
0.35328946
0.9665695
0.94139796
0.9916843
0.94406515
0.99657744
0.69188833
0.9997192
1.0
1.0
0.7780048
0.0319146
0.99686193
0.958462
0.9368966
0.97950715
0.736

In [17]:
pred_lbl

[1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,


In [18]:
# pred_lbl_flattened = [val for sublist in pred_lbl for val in sublist]

In [19]:
# pred_lbl_flattened

In [20]:
labels

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,


In [21]:
wrong_ls=[]
for i in range(len(pred_lbl)):
    # print(pred_lbl[i])
    # print(labels[i])
    if (pred_lbl[i] !=labels[i]):
        wrong_ls.append(i)

In [22]:
wrong_ls

[0,
 2,
 5,
 6,
 8,
 10,
 12,
 14,
 16,
 20,
 22,
 25,
 26,
 30,
 34,
 36,
 40,
 42,
 44,
 48,
 54,
 55,
 56,
 58,
 60,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 80,
 84,
 88,
 90,
 94,
 96,
 98,
 100,
 102,
 103,
 104,
 106,
 108,
 114,
 116,
 118,
 120,
 122,
 124,
 126,
 128,
 130,
 132,
 136,
 144,
 148,
 149,
 150,
 152,
 154,
 156,
 157,
 158,
 160,
 164,
 170,
 174,
 176,
 178,
 180,
 182,
 184,
 186,
 192,
 194,
 195,
 196,
 198,
 202,
 206,
 208,
 210,
 212,
 216,
 220,
 222,
 224,
 226,
 228,
 234,
 236,
 242,
 246,
 254,
 258,
 260,
 262,
 265,
 266,
 267,
 268,
 274,
 280,
 282,
 284,
 286,
 288,
 296,
 300,
 303,
 304,
 308,
 310,
 314,
 317,
 318,
 320,
 322,
 324,
 326,
 328,
 329,
 330,
 336,
 338,
 347,
 348,
 350,
 352,
 362,
 364,
 366,
 368,
 370,
 372,
 374,
 376,
 378,
 380,
 382,
 383,
 384,
 386,
 388,
 392,
 394,
 396,
 400,
 404,
 406,
 411,
 412,
 413,
 414,
 416,
 420,
 422,
 424,
 426,
 428,
 430,
 431,
 432,
 434,
 438,
 440,
 446,
 448,
 450,
 452,
 453,
 454,


In [23]:
len(wrong_ls)

518

In [26]:
(1320-len(wrong_ls))/1320

0.6075757575757575

In [27]:
len(test_generator.labels)

1320

In [28]:
photos = []
for item in wrong_ls:
    # print(item)
    photos.append(test_generator.labels[item])

In [29]:
len(photos)

518

In [30]:
photos

[['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp',
  0,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  0,
  70,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-12.bmp',
  '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/M-048/M-048-08.bmp',
  1,
  224,
  224],
 ['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2mar

In [31]:
photos[0][0]

'/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-16.bmp'

In [32]:
photos[0][1]

'/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/CLSPOOLER_2margin_full_data_3hrs/data_testset/W-048/W-048-24.bmp'

In [42]:
scarf = 0
sunglasses = 0
normal = 0
yawn = 0
for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        scarf+=1
    if root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        scarf+=1
    if root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        scarf+=1
    if root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        scarf+=1

    # SUNGLASSES
    if root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        sunglasses+=1
    if root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        sunglasses+=1
    if root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        sunglasses+=1
    if root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        sunglasses+=1
    
    # YAWN
    if root1.endswith("04") or root1.endswith("17"):
        yawn+=1
    if root2.endswith("04") or root2.endswith("17"):
        yawn+=1


In [43]:
print(f"Scarf Total: {scarf} Percentage: {(scarf/(len(photos)*2))*100}")
print(f"Sunglasses Total: {sunglasses} Percentage: {(sunglasses/(len(photos)*2))*100}")
print(f"Yawn Total: {yawn} Percentage: {(yawn/(len(photos)*2))*100}")
print(f"Others: {(391*2)-scarf-sunglasses-yawn} Percentage: {(((len(photos)*2)-scarf-sunglasses-yawn)/(len(photos)*2))*100}")

Scarf Total: 265 Percentage: 25.57915057915058
Sunglasses Total: 244 Percentage: 23.552123552123554
Yawn Total: 73 Percentage: 7.046332046332046
Others: 200 Percentage: 43.82239382239382


In [44]:
scarf = 0
sunglasses = 0
normal = 0
yawn = 0
for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        scarf+=1
    elif root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        scarf+=1
    elif root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        scarf+=1
    elif root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        scarf+=1

    # SUNGLASSES
    if root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        sunglasses+=1
    elif root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        sunglasses+=1
    elif root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        sunglasses+=1
    elif root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        sunglasses+=1
    
    # YAWN
    if root1.endswith("04") or root1.endswith("17"):
        yawn+=1
    elif root2.endswith("04") or root2.endswith("17"):
        yawn+=1

In [45]:
print(f"Scarf Total: {scarf} Percentage: {(scarf/(len(photos)))*100}")
print(f"Sunglasses Total: {sunglasses} Percentage: {(sunglasses/(len(photos)))*100}")
print(f"Yawn Total: {yawn} Percentage: {(yawn/(len(photos)))*100}")

Scarf Total: 239 Percentage: 46.13899613899614
Sunglasses Total: 214 Percentage: 41.312741312741316
Yawn Total: 70 Percentage: 13.513513513513514


In [46]:
obstruction =0
normal = 0

for item in photos:
    root1, ext1 = os.path.splitext(item[0])
    root2, ext2 = os.path.splitext(item[1])

    # SCARF
    if root1.endswith("11") or root1.endswith("12") or root1.endswith("13"):
        obstruction+=1
    elif root2.endswith("11") or root2.endswith("12") or root2.endswith("13"):
        obstruction+=1
    elif root1.endswith("24") or root1.endswith("25") or root1.endswith("26"):
        obstruction+=1
    elif root2.endswith("24") or root2.endswith("25") or root2.endswith("26"):
        obstruction+=1
    # SUNGLASSES
    elif root1.endswith("08") or root1.endswith("09") or root1.endswith("10"):
        obstruction+=1
    elif root2.endswith("08") or root2.endswith("09") or root2.endswith("10"):
        obstruction+=1
    elif root1.endswith("21") or root1.endswith("22") or root1.endswith("23"):
        obstruction+=1
    elif root2.endswith("21") or root2.endswith("22") or root2.endswith("23"):
        obstruction+=1
    # YAWN
    elif root1.endswith("04") or root1.endswith("17"):
        obstruction+=1
    elif root2.endswith("04") or root2.endswith("17"):
        obstruction+=1
    else:
        normal+=1

In [47]:
print(f"Obstruction Total: {obstruction} Percentage: {(obstruction/(len(photos)))*100}")
print(f"Normal Total: {normal} Percentage: {(normal/(len(photos)))*100}")

Obstruction Total: 431 Percentage: 83.20463320463321
Normal Total: 87 Percentage: 16.795366795366796


In [48]:
scarf_vs_scarf = 0
scarf_vs_sunglasses = 0
scarf_vs_normal = 0
sunglasses_vs_normal = 0
yawn_vs_yawn = 0

In [49]:
home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_testset_nocover')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [50]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'test': np.arange(math.floor(len(pairs)))}

# Generators
# test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=16)
test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=1)

In [51]:
print(f"Number of Test Pairs: {len(partition['test'])}")

Number of Test Pairs: 608


In [52]:
loss, acc = siamese_network.evaluate(test_generator)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

608/608 [==============================] - 33s 55ms/step - loss: 0.9139 - accuracy: 0.6431
Restored model, accuracy: 64.31%
